In [ ]:
#@title 1. Замена лица на видео
#@markdown   Использовать гугл диск:
use_gdrive = True #@param {type:"boolean"}
#@markdown   Укажите путь до файла изображения с новым лицом:
use_this_face = "" #@param {type:"string"}
#@markdown   Укажите путь до файла с видео:
replace_this_face = "" #@param {type:"string"}
#@markdown   Сохранить папку с изображениями раскадровки:
keep_frames_directory = False #@param {type:"boolean"}
#@markdown   Поменять лица у всех на видео
all_faces = False #@param {type:"boolean"}
#@markdown   <br> Если вы используете гугл диск, то готовое видео будет сохранено на гугл диске в папке roop/swaped_output.mp4

_kf_dir = ""
_output = ""
_all_faces = ""

from google.colab import drive
import os

if keep_frames_directory:
    _kf_dir = "--keep-frames"
else:
    _kf_dir = ""

if all_faces:
    _all_faces = "--all-faces"
else:
    _all_faces = ""

if use_gdrive:
    drive.mount('/content/drive')
    if not os.path.exists('/content/drive/MyDrive/roop'):
        !mkdir -p /content/drive/MyDrive/roop
    _output = "-o /content/drive/MyDrive/roop/swaped_output.mp4"
else:
    _output = ""

!git clone https://github.com/s0md3v/roop
%cd /content/roop
!pip install -r requirements.txt

if not os.path.exists('/content/roop/inswapper_128.onnx'):
    !wget -O /content/roop/inswapper_128.onnx 'https://civitai.com/api/download/models/85159?type=Model&format=Other&size=full&fp=fp32'

%cd /content/roop
!python run.py -f $use_this_face -t $replace_this_face $_output --gpu $_kf_dir $_all_faces